In [1]:
import os


In [2]:

%pwd

'c:\\Users\\dixit\\Desktop\\MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\dixit\\Desktop\\MLOPS'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.Classifier_Model.constants import *
from src.Classifier_Model.utils.common import read_yaml, create_directories

In [7]:
print(CONFIG_FILE_PATH)

C:\Users\dixit\Desktop\MLOPS\config\config.yaml


In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        Config = self.config.data_ingestion

        create_directories([Config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Config.root_dir,
            source_URL=Config.source_URL,
            local_data_file=Config.local_data_file,
            unzip_dir=Config.unzip_dir
        )

        return data_ingestion_config
    



In [9]:
import os
import zipfile
import gdown
from src.Classifier_Model import logger
from src.Classifier_Model.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

print(CONFIG_FILE_PATH)
print(CONFIG_FILE_PATH.exists())
print(PARAMS_FILE_PATH.exists())
print(CONFIG_FILE_PATH.resolve())


C:\Users\dixit\Desktop\MLOPS\config\config.yaml
True
True
C:\Users\dixit\Desktop\MLOPS\config\config.yaml


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-03-16 21:22:15,230: INFO: common: yaml file: C:\Users\dixit\Desktop\MLOPS\config\config.yaml loaded successfully]
[2024-03-16 21:22:15,232: INFO: common: yaml file: C:\Users\dixit\Desktop\MLOPS\params.yaml loaded successfully]
[2024-03-16 21:22:15,233: INFO: common: created directory at: artifacts]
[2024-03-16 21:22:15,234: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-16 21:22:15,235: INFO: 2712413423: Downloading data from https://drive.google.com/file/d/1sBCJ41PFRUUGMRs6b9eh1q7fQp4-1Yc5/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1sBCJ41PFRUUGMRs6b9eh1q7fQp4-1Yc5
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1sBCJ41PFRUUGMRs6b9eh1q7fQp4-1Yc5&confirm=t&uuid=f83cf206-0875-42b1-94e4-3b7ae9a9e5a2
To: c:\Users\dixit\Desktop\MLOPS\artifacts\data_ingestion\data.zip
100%|██████████| 939M/939M [00:32<00:00, 29.3MB/s] 

[2024-03-16 21:22:49,236: INFO: 2712413423: Downloaded data from https://drive.google.com/file/d/1sBCJ41PFRUUGMRs6b9eh1q7fQp4-1Yc5/view?usp=sharing into file artifacts/data_ingestion/data.zip]
